# SQLAlchemy Joins

## Setup

In [34]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect

In [35]:
engine = create_engine("sqlite:///../Resources/mammal_masses.sqlite", echo=False)

In [36]:
# Reflect Database into ORM classes
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()

['ea', 'na']

In [38]:
# Map Europe class
EA = Base.classes.ea

In [39]:
# Map North American class
NA = Base.classes.na

In [40]:
# create a session
session = Session(engine)

## Filtering Review

Filters are the "WHERE" clause for your select statement. 

In [41]:
# filter North American mammals whose genus is "Antilocapra"
# query, loop over and print out animals.
mammals = session.query(NA).filter(NA.genus == 'Antilocapra').all()
for mammal in mammals:
    print("Family: {0}, Genus: {1}".format(mammal.family, mammal.genus))

Family: Antilocapridae, Genus: Antilocapra


In [43]:
mamals = engine.execute(''' select *
from na
where genus = 'Antilocapra' 
limit 10
''').fetchall()

[(1, 3898, None, 'extant', 'Artiodactyla', 'Antilocapridae', 'Antilocapra', 'americana', 4.66, 46082.9, '60')]

## Joins

In [44]:
inspector = inspect(engine)
inspector.get_table_names()

['ea', 'na']

In [45]:
# Get a list of column names and types
columns = inspector.get_columns('ea')
for c in columns:
    print(c['name'], c["type"])

id INTEGER
record_id INTEGER
continent TEXT
status TEXT
sporder TEXT
family TEXT
genus TEXT
species TEXT
log_mass_g FLOAT
comb_mass_g FLOAT
reference TEXT


In [47]:
session.query(EA.sporder, NA.sporder).distinct().limit(10).all()

[('Artiodactyla', 'Artiodactyla'),
 ('Artiodactyla', 'Carnivora'),
 ('Artiodactyla', 'Chiroptera'),
 ('Artiodactyla', 'Didelphimorphia'),
 ('Artiodactyla', 'Insectivora'),
 ('Artiodactyla', 'Lagomorpha'),
 ('Artiodactyla', 'Perissodactyla'),
 ('Artiodactyla', 'Primates'),
 ('Artiodactyla', 'Proboscidea'),
 ('Artiodactyla', 'Rodentia')]

### Using Filter as Joins

In [48]:
same_sporder = session.query(EA, NA).filter(EA.sporder == NA.sporder).limit(10).all()


for record in same_sporder:
    (ea, na) = record
    print(f"{ea.sporder} , {na.sporder}")

Artiodactyla , Artiodactyla
Artiodactyla , Artiodactyla
Artiodactyla , Artiodactyla
Artiodactyla , Artiodactyla
Artiodactyla , Artiodactyla
Artiodactyla , Artiodactyla
Artiodactyla , Artiodactyla
Artiodactyla , Artiodactyla
Artiodactyla , Artiodactyla
Artiodactyla , Artiodactyla


In [50]:
# Return all animals from EA and NA belonging to the same sporder.
# This JOINs the data in the two tables together into a single dataset (here in the form of a tuple).
# Note: We are going to limit the results to 10 for printing
sel = [EA.family, EA.genus, EA.species, NA.family, NA.genus, NA.species]
same_sporder = session.query(*sel).filter(EA.sporder == NA.sporder).limit(10).all()

q = engine.execute('''select ea.family, ea.genus, ea.species, na.family, na.genus, na.species
                from ea  
                inner join na on na.sporder = ea.sporder
                limit 10

''')

for record in same_sporder:
    (ea_fam, ea_gen, ea_spec, na_fam, na_gen, na_spec) = record
    print(
        f"The European animal '{ea_fam} {ea_gen} {ea_spec}'"
        f"belongs to the same sporder as the North American animal '{na_fam} {na_gen} {na_spec}'.")

The European animal 'Bovidae Antilope cervicapra'belongs to the same sporder as the North American animal 'Antilocapridae Antilocapra americana'.
The European animal 'Bovidae Antilope cervicapra'belongs to the same sporder as the North American animal 'Antilocapridae Capromeryx mexicana'.
The European animal 'Bovidae Antilope cervicapra'belongs to the same sporder as the North American animal 'Antilocapridae Capromeryx minor'.
The European animal 'Bovidae Antilope cervicapra'belongs to the same sporder as the North American animal 'Antilocapridae Stockoceros conklingi'.
The European animal 'Bovidae Antilope cervicapra'belongs to the same sporder as the North American animal 'Antilocapridae Stockoceros onusrosagris'.
The European animal 'Bovidae Antilope cervicapra'belongs to the same sporder as the North American animal 'Antilocapridae Tetrameryx shuleri'.
The European animal 'Bovidae Antilope cervicapra'belongs to the same sporder as the North American animal 'Bovidae Bison bison'.
Th

In [56]:
for row in q:
     (ea_fam, ea_gen, ea_spec, na_fam, na_gen, na_spec) = row
    print(record)
#     print(
#         f"The European animal '{ea_fam} {ea_gen} {ea_spec}'"
#         f"belongs to the same sporder as the North American animal '{na_fam} {na_gen} {na_spec}'.")

('Bovidae', 'Bubalus', 'mephistopheles', 'Antilocapridae', 'Antilocapra', 'americana')
('Bovidae', 'Bubalus', 'mephistopheles', 'Antilocapridae', 'Antilocapra', 'americana')
('Bovidae', 'Bubalus', 'mephistopheles', 'Antilocapridae', 'Antilocapra', 'americana')
('Bovidae', 'Bubalus', 'mephistopheles', 'Antilocapridae', 'Antilocapra', 'americana')
('Bovidae', 'Bubalus', 'mephistopheles', 'Antilocapridae', 'Antilocapra', 'americana')
('Bovidae', 'Bubalus', 'mephistopheles', 'Antilocapridae', 'Antilocapra', 'americana')
('Bovidae', 'Bubalus', 'mephistopheles', 'Antilocapridae', 'Antilocapra', 'americana')
('Bovidae', 'Bubalus', 'mephistopheles', 'Antilocapridae', 'Antilocapra', 'americana')
('Bovidae', 'Bubalus', 'mephistopheles', 'Antilocapridae', 'Antilocapra', 'americana')
('Bovidae', 'Bubalus', 'mephistopheles', 'Antilocapridae', 'Antilocapra', 'americana')
